## Libraries

In [ ]:
#!pip install fastDamerauLevenshtein
#!pip install fuzzywuzzy
#!pip install fuzzymatcher
#!pip install sparse_dot_topn 
#!pip install editdistance
#!pip install fastDamerauLevenshtein

In [ ]:
from nltk import ngrams

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import re

In [ ]:
from fuzzywuzzy import process, fuzz
#from fastDamerauLevenshtein import damerauLevenshtein

In [ ]:
import difflib
from difflib import SequenceMatcher, get_close_matches



#if-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from nltk.util import ngrams
#import nltk.util.ngrams

#Kavya
import editdistance

# 1 Data Pre-processing

## Exploratory Analysis

### 1.1 Read Data

In [2]:
df=pd.read_excel('Aliasing_Master.xlsx') #Fix mention sheet name

In [ ]:
df = pd.read_csv('Aliasing_Master.csv',skipinitialspace = True)

In [ ]:
# reading csv file and at a same time using converters attribute which will remove extra space
df = pd.read_csv('Aliasing_Master.csv', converters={'PID': str.strip(),
                                                'SPI' : str.strip(),
                                                'PDS' : str.strip()})

### 1.2 Data Overview

In [ ]:
df.head(3)

The features have different syntaxes across columns. White Spaces, seperators like '-' are present in the data.

In [ ]:
df.tail(3)

In [3]:
df.drop(['Unnamed: 3',	'Alias',	'Master'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe(include='object')

'PID', 'SPI', 'PDS' columns have Unique items and there are no duplicates.

### 1.3 Data Pre-processing - Features

In [4]:
for i in df.columns:
    df[i] = df[i].str.strip()

In [5]:
for feat in df.columns:
    words_list = []
    alpha_words_list = []
    numeric_words_list = []
    
    for x in df[feat]:
        if pd.notnull(x):
            words = re.findall(r'[A-Za-z]+|\d+', str(x))
            alpha_words = [word for word in words if word.isalpha()]
            numeric_words = [word for word in words if word.isdigit()]
            
            words_list.append(words)
            alpha_words_list.append(alpha_words)
            numeric_words_list.append(numeric_words)
    
    df[f'{feat}_bag_of_all'] = words_list
    df[f'{feat}_bag_of_alpha'] = alpha_words_list
    df[f'{feat}_bag_of_num'] = numeric_words_list


ValueError: Length of values (14299) does not match length of index (56420)

In [15]:
for feat in df.columns:
    words_list = []
    alpha_words_list = []
    numeric_words_list = []

    for x in df[feat]:
        if len(x) > 0:  # Check if not null and not empty
            words = re.findall(r'[A-Za-z]+|\d+', str(x))
            alpha_words = [word for word in words if word.isalpha()]
            numeric_words = [word for word in words if word.isdigit()]

            words_list.append(words)
            alpha_words_list.append(alpha_words)
            numeric_words_list.append(numeric_words)

    if len(words_list) > 0:
        df[f'{feat}_bag_of_all'] = pd.Series(words_list, index=df.index[:len(words_list)])
        df[f'{feat}_bag_of_alpha'] = pd.Series(alpha_words_list, index=df.index[:len(alpha_words_list)])
        df[f'{feat}_bag_of_num'] = pd.Series(numeric_words_list, index=df.index[:len(numeric_words_list)])


TypeError: object of type 'float' has no len()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
spid_unique_df = pd.DataFrame()
spi_unique_df = pd.DataFrame()
pds_unique_df = pd.DataFrame()

for i in df.columns:

    #print('{0} Column Count is {1}'.format(i, df[i].count()))

    if i == 'PID':
        spid_unique_list = df['PID'].unique().tolist()
        spid_unique_df[i] = spid_unique_list
    elif i == 'SPI':
        spi_unique_list = df['SPI'].unique().tolist()
        spi_unique_df[i] = spi_unique_list
    elif i == 'PDS':
        pds_unique_list = df['PDS'].unique().tolist()
        pds_unique_df[i] = pds_unique_list

### 1.4 Unique DF

In [ ]:
unique_df= pd.DataFrame()
unique_list=[]

for i in df.columns:
    
    unique_list = df[i].unique().astype('str').tolist()
    unique_temp_df = pd.DataFrame(unique_list, dtype= 'str',columns =[i])
    unique_df=pd.concat([unique_df,unique_temp_df],axis=1)
    #exec('df_{} = pd.DataFrame()'.format(i))
    #exec('df_{}'.format(i)[i]) = pd.DataFrame(unique_list, columns =i)
    
    #print('{0} Column Count is {2}. Unique Tags are: {1}'.format(i,len(unique_list)-1, df[i].count()))
    #print(' ')
          
#print(unique_df.dtypes)
#unique_df.head(3)

In [ ]:
print('Unique Dataset shape:', unique_df.shape)
unique_df.head(3)

### 1.5 Special Characters

In [ ]:
import re

text = "This is a string with whitespace and special characters !@#$%^&*()_+"
clean_text = re.sub(r'\W+', '', str(text))

print(clean_text)

In [ ]:
unique_df = unique_df.replace('\s+', ' ', regex=True)
# apply the function to each element in the dataframe

unique_df = unique_df.replace('-', ' ', regex=True)

# print the cleaned dataframe
unique_df.head(3)

In [ ]:
## define a function to remove whitespace and special characters from a string
#def clean_string(s):
    #return re.sub(r'\W+', '', str(s))

In [ ]:
## apply the function to each element in the dataframe
#df = df.applymap(clean_string)

## print the cleaned dataframe
#df.head(3)

In [ ]:
### Future Scope, collect the '-' Index/position in each string

### 1.6 Spaces

In [ ]:
#for i in df.columns:
 #   df[i].str.strip()

In [ ]:
#df.head(3)

### 1.7 Sort Features

### 1.8 Small Dataset

In [ ]:
spid_unique = unique_df['PID'].unique().tolist()
spi_unique = unique_df['SPI'].unique().tolist()
pds_unique = unique_df['PDS'].unique().tolist()

In [ ]:
spid_small=spid_unique[:3000]
spi_small=spi_unique[:3000]
pds_small=pds_unique[:300]

In [ ]:
### 1.5 Future Scope - Bucketing/Binning into Patterns

In [ ]:
### 1.5 Future Scope - 2 Dimensional Array of Alphabet & Numbers

## 2 Clustering

In [ ]:
import scipy.cluster.hierarchy as hc
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

In [ ]:
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Convert texts to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()

In [ ]:
from scipy.cluster.hierarchy import fcluster
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster.hierarchy import dendrogram, linkage

# Create the document-term matrix
vectorizer = CountVectorizer()

### 2.1 K-Means Clustering: 'PID'

In [ ]:
X = vectorizer.fit_transform(spid_unique_df['PID'].values.astype('U'))

In [ ]:
wss =[] 
for i in range(1,20):
    KM = KMeans(n_clusters=i)
    KM.fit(X)
    wss.append(KM.inertia_)

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(range(1,20), wss, marker='o', markerfacecolor='red', markersize=10)
plt.xlabel('K (Number of Clusters)')
plt.ylabel('Inertia Score')
plt.title('Inertia vs K Value')

In [ ]:
# Cluster the texts using K-Means algorithm
kmeans = KMeans(n_clusters=8)
kmeans.fit(X)

labels = kmeans.labels_
spid_unique_df['PID_kmeans'] = labels

# Print the cluster labels assigned to each text
#for i, pid in enumerate(spid_unique):
 #   print(f"Text {i+1} is assigned to cluster {kmeans.labels_[i]+1}")

### 2.2 Hierarchial Clustering: 'PID'

In [ ]:
# Create the document-term matrix

dtm = vectorizer.fit_transform(spid_unique_df['PID'].values.astype('U'))
# Perform hierarchical clustering
Z = linkage(dtm.toarray(), method='ward')

In [ ]:
# Create a dendrogram
plt.figure(figsize=(10, 5))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(Z, leaf_rotation=90., leaf_font_size=8.)
plt.show()


In [ ]:
#wardlink = linkage(dtm.toarray(), method = 'ward')
#Show last p  merges, setting p-10, Fix- metric
dend = dendrogram(Z, truncate_mode='lastp', p=15)

In [ ]:

#Assign cluster to records using fcluster
#Method1
clusters = fcluster(Z, 23, criterion='distance')

In [ ]:
spid_unique_df['PID_clusters'] = clusters
spid_unique_df.head(3)

In [ ]:
spid_unique_df.to_csv('pid_clust.csv')

### 2.3 Hierarchial Clustering : 'SPI'

In [ ]:
# Create the document-term matrix

dtm = vectorizer.fit_transform(spi_unique_df['SPI'].values.astype('U'))
# Perform hierarchical clustering
Z = linkage(dtm.toarray(), method='ward')

# Create a dendrogram
plt.figure(figsize=(10, 5))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(Z, leaf_rotation=90., leaf_font_size=8.)
plt.show()

Each branch in the 'SPI' clusters are clean and 7 is a good cluster

In [ ]:
#Assign cluster to records using fcluster
#Method1
clusters = fcluster(Z, 7, criterion='maxclust')

spi_unique_df['SPI_clusters'] = clusters

In [ ]:
spi_unique_df.to_csv('spi_clusters.csv')

### 2.5 KMeans Clustering : 'PDS'

In [ ]:
X = vectorizer.fit_transform(pds_unique_df['PDS'].values.astype('U'))

In [ ]:
# Cluster the texts using K-Means algorithm

wss =[] 
for i in range(1,20):
    KM = KMeans(n_clusters=i, random_state=123)
    KM.fit(X)
    wss.append(KM.inertia_)

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(range(1,20), wss, marker='o', markerfacecolor='red', markersize=10)
plt.xlabel('K (Number of Clusters)')
plt.ylabel('Inertia Score')
plt.title('Inertia vs K Value')

In [ ]:
# Cluster the texts using K-Means algorithm
kmeans = KMeans(n_clusters=10, random_state=123)
kmeans.fit(X)

In [ ]:
labels = kmeans.labels_
pds_unique_df['PDS_kmeans'] = labels

pds_unique_df.head(3)

### 2.6 Hierrachial Clustering : 'PDS'

In [ ]:
# Create the document-term matrix

dtm = vectorizer.fit_transform(pds_unique_df['PDS'].values.astype('U'))
# Perform hierarchical clustering
Z = linkage(dtm.toarray(), method='ward')

# Create a dendrogram
plt.figure(figsize=(10, 5))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(Z, leaf_rotation=90., leaf_font_size=8.)
plt.show()

Each branch in the 'PDS' clusters are clean and 7 is a good cluster

In [ ]:
#Assign cluster to records using fcluster
#Method1
clusters = fcluster(Z, 7, criterion='maxclust')

pds_unique_df['PDS_clusters'] = clusters

In [ ]:
pds_unique_df.to_csv('pds_clusters.csv')

Assumption now observation after SPID and Instr Bidirectional test:

PID is master left column.

Direction of the search is PID searched in SPI, PID is serached in PDS

Best Match for PDS

# 2 Models

## 2.1 FuzzyWuzzy

In [ ]:
#Query model
#results = process.extract(query, choices, scorer=fuzz.token_sort_ratio)

### 2.1.1 Algorithm 0 -FuzzyWuzzy  Instr Tag in SPID

In [ ]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in spi_small
             for i in process.extract(x, spid_small, scorer=fuzz.token_sort_ratio)]

In [ ]:
#Create dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['spi_sort','spid_sort','score_sort'])

In [ ]:
cutoff= 81
similarity_sort['similarity_flag'] = (similarity_sort['score_sort'] > cutoff).astype(int)

In [ ]:
# x in spid column.
similarity_sort.head(50)

In [ ]:
similarity_sort.to_csv('Fuzzy_10k_Instr_in_SPID.csv')

In [ ]:
similarity_sort.head(15)

### 2.1.2 Algorithm 1- FuzzyWuzzy SPID Tag in Instr

In [ ]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in spid_small
             for i in process.extract(x, spi_small, scorer=fuzz.token_sort_ratio)]

In [ ]:
#Create dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['spid_sort','spi_sort','score_sort'])

In [ ]:
cutoff= 80
similarity_sort['similarity_flag'] = (similarity_sort['score_sort'] >= cutoff).astype(int)

In [ ]:
# x in Instr column.
similarity_sort.head(50)

In [ ]:
#Derive representative values
similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['spi_sort'], similarity_sort['spid_sort'])

In [ ]:
similarity_sort.head(50)

### 2.1.3 SPID in PDS

In [ ]:
#Create tuples of brand names, matched brand names, and the score
#(spid_unique_df['PID'].values.astype('U')
score_sort = [(x,) + i
             for x in unique_df['PDS'].values.astype('U')
             for i in process.extract(x, spid_small, scorer=fuzz.token_sort_ratio)]

In [ ]:
#Create dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=[ 'pds_sort', 'spid_sort','score_sort'])

In [ ]:
cutoff= 81
similarity_sort['similarity_flag'] = (similarity_sort['score_sort'] > cutoff).astype(int)

In [ ]:
similarity_sort.to_csv('Fuzzy_10k_PDS_in_SPID.csv')

In [ ]:
similarity_sort.head(15)

## 2.2 Algorithm 1- Knuth Morris - Sivaji

## 2.3 Algorithm 2- Boyer-Moore

https://cmps-people.ok.ubc.ca/ylucet/DS/BoyerMoore.html

https://www.cs.utexas.edu/users/moore/best-ideas/string-searching/index.html

https://www.geeksforgeeks.org/boyer-moore-algorithm-for-pattern-searching/````

## 2.4 Algorithm 3- Damerau Levenstein

similarity:

If this parameter value is False, it will return the total cost of edit, otherwise it will return a score from 0.0 to 1.0 denoting how similar the two objects are. It is True by default

In [ ]:
from fastDamerauLevenshtein import damerauLevenshtein

In [ ]:
damerauLevenshtein('ca', 'abc', similarity=False)  # expected result: 2.0
damerauLevenshtein('car', 'cars', similarity=True)  # expected result: 0.75
damerauLevenshtein(['ab', 'bc'], ['ab'], similarity=False)  # expected result: 1.0
damerauLevenshtein(['ab', 'bc'], ['ab'], similarity=True)  # expected result: 0.5

In [ ]:
# https://www.geeksforgeeks.org/damerau-levenshtein-distance/
    
def optimal_string_alignment_distance(s1, s2):
    # Create a table to store the results of subproblems
    dp = [[0 for j in range(len(s2)+1)] for i in range(len(s1)+1)]
     
    # Initialize the table
    for i in range(len(s1)+1):
        dp[i][0] = i
    for j in range(len(s2)+1):
        dp[0][j] = j
 
    # Populate the table using dynamic programming
    for i in range(1, len(s1)+1):
        for j in range(1, len(s2)+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
 
    # Return the edit distance
    return dp[len(s1)][len(s2)]
 
print(optimal_string_alignment_distance("geeks", "forgeeks"))

In [ ]:
x= 'JE-CRC301-FE-80'
y = 'JE-CRC301-FE-801'
lev_dist = optimal_string_alignment_distance(x, y)
lev_dist

In [ ]:
#Create tuples of brand names, matched brand names, and the score
for x in spid_small:
    for y in spi_small:
        print(optimal_string_alignment_distance(x, y))
    

In [ ]:
#https://pypi.org/project/editdistance/

import editdistance
editdistance.eval('banana', 'bahama')

The challenge is that these algorithms (e.g. Levenshtein, Damerau-Levenshtein, Jaro-Winkler, q-gram, cosine) are computationally intensive. Trying to do a lot of matching on large data sets is not scaleable.

## 2.5 Entity Resolution

https://www.districtdatalabs.com/basics-of-entity-resolution

## 2.6 tf-idf 

https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#from scipy.sparse import csr_matrix

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#from scipy.sparse import csr_matrix
#!pip install sparse_dot_topn 
#import sparse_dot_topn.sparse_dot_topn as ct

org_names = df['PID'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(org_names)

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
return csr_matrix((data,indices,indptr),shape=(M,N))

## 2.7 Kavya

In [ ]:
import pandas as pd
import editdistance
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

In [ ]:
!pip install --upgrade editdistance

In [ ]:
from Levenshtein import distance

In [ ]:
data = pd.read_excel('Aliasing_Master.xlsx')
column1 = data['PID'].astype(str).tolist()
column2 = data['SPI'].astype(str).tolist()

In [ ]:
threshold = 3
matches = []
for i in range(len(column1)):
    for j in range(len(column2)):
        if editdistance.distance(column1[i], column2[j]) <= threshold:
            matches.append((i, j))

In [ ]:
!pip install editdistance

import editdistance

str1 = "example"
str2 = "samples"

distance = editdistance.damerau_levenshtein_distance(str1, str2)
print(distance)  # Output: 3


In [ ]:

wb = Workbook()
ws = wb.active
for i, j in matches:
    ws[f'A{i+1}'] = column1[i]
    ws[f'B{i+1}'] = column2[j]

 

wb.save('matched_pairs.xlsx')

## 2.8 Fast Text

https://github.com/nahida-uap/Text-Similarity-Metrics-in-Python/blob/master/fastText-embedding-model-soft-cosine-similarity.py

In [ ]:
#import require libraries
import gensim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess

#print(gensim.__version__)

# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

#Define the documents
rev1 = "Overall the app is very good. I always use it. But after the Android 10 update on the Google pixel 2 xl, I'm not able to chat with other contacts while I'm on video call with picture to picture mode like before. Emailed support and it's like they didn't even try to understand what the problem was and just replied with random links resolutions. Not sure if it was Android update issue or what. Still have the issue."

rev2 = "This would be a much better app if it wasn't such a pain in the arse to update contacts. How hard does it have to be? *Edit*. A year on, and still a pain. Trying to get a contact who is on WhatsApp into the contact list just doesn't happen. You can refresh until you're blye in the face. It still wont add them."

rev3 = "A Feature request: so my whatsapp media is not saved to my phone gallery because most of them are memes , greetings,news but sometimes it's Important photos/videos which i want to save in gallery and there is no option to save this photo to gallery . Please add this feature it would be very helpfull"

rev4 = "The what's new is a lie. It's said the same thing for multiple updates. Why the secretive updates? What are you doing to our devices? // 20 Aug another secretive update with false changelog // 3 Sep *another* update delivered with a FALSE update. This is unaceptable. This is digital rape."

rev5 = "Hii..... WhatsApp My model is motog4 plus ....not show fingerprint lock option why please answer dedo bhai in Hindi????"

rev6 = "We need to have improvements on the quality phone conversations. No smooth flow of conversation, and too many reconnecting instances in one chat, or Tel conversation."

documents = [rev1, rev2, rev3, rev4, rev5, rev6]


#Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

#Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

#Embeddings - Convert the sentences into bag-of-words vectors.
sent_1 = dictionary.doc2bow(simple_preprocess(rev1))
sent_2 = dictionary.doc2bow(simple_preprocess(rev2))
sent_3 = dictionary.doc2bow(simple_preprocess(rev3))
sent_4 = dictionary.doc2bow(simple_preprocess(rev4))
sent_5 = dictionary.doc2bow(simple_preprocess(rev5))
sent_6 = dictionary.doc2bow(simple_preprocess(rev6))

sentences = [sent_1, sent_2, sent_3, sent_4, sent_5, sent_6]


#Compute soft cosine similarity of 2 documents
#print(softcossim(sent_1, sent_2, similarity_matrix))

#Compute soft cosine similarity matrix
import numpy as np
import pandas as pd

def soft_cosine_similarity_matrix(sentences):
    len_array = np.arange(len(sentences))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity_matrix) ,2) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

#print(documents)
soft_cosine_similarity_matrix(sentences)

# 3 Model Evaluation

## 3.1 TDIFDVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Define the two strings to compare
s1 = "C301FE-150007"
s2 = 'JE-CRC301-FE-150007'

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the documents and transform the documents into TF-IDF vectors
vectors = vectorizer.fit_transform([s1, s2])

# Compute the cosine similarity between the two vectors
similarity = cosine_similarity(vectors[0], vectors[1])[0][0]

print("The similarity score between the two strings is:", similarity)

In [ ]:
X = vectorizer.fit_transform(spid_unique_df['PID'].values.astype('U'))

In [ ]:
condition = similarity_sort['similarity_flag'] == 1
filtered_df = similarity_sort[condition]
print(filtered_df.shape)
filtered_df.head(3)

In [ ]:
filtered_df.drop(['score_sort','similarity_flag'],axis=1).values.astype('U')

In [ ]:
filtered_df

In [ ]:
eval_df= pd.DataFrame(filtered_df, columns=['spi_sort', 'spid_sort'])

In [ ]:
eval_df.head(3)

In [ ]:
# Compute the cosine similarity matrix
#spid_unique_df['PID'].values.astype('U')
cosine_sim = cosine_similarity(eval_df.values.astype('U'))

In [ ]:
print(type(cosine_sim))

In [ ]:
for s1 in spid_unique:
    

In [ ]:
#To use cosine similarity on two string features in a dataframe, you can follow these steps:

#Load the necessary libraries: pandas, numpy, and scikit-learn.

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#Create a pandas dataframe with the two string features you want to compare. For example:

data = similarity_score
df = pd.DataFrame(data)
#Instantiate a CountVectorizer object to transform the string data into a bag-of-words model, where each word is represented by a feature in a vector. This will be used to calculate the cosine similarity.

vectorizer = CountVectorizer().fit_transform(df['string1'] + df['string2'])
#Split the vectorized data into two separate sets for the two string features you want to compare.

vectorizer1 = vectorizer[:len(df)]
vectorizer2 = vectorizer[len(df):]
#Calculate the cosine similarity matrix between the two sets of vectorized data using cosine_similarity function from scikit-learn.

similarity_matrix = cosine_similarity(vectorizer1, vectorizer2)
#Store the cosine similarity matrix in a new dataframe with rows labeled with the indices of string1 and columns labeled with the indices of string2.

similarity_df = pd.DataFrame(similarity_matrix, index=df.index, columns=df.index)
#Display the resulting similarity dataframe to compare the two string features.

print(similarity_df)
#This will output a square dataframe where each cell represents the cosine similarity between the string data at the corresponding row and column indices.







## 3.2 Fuzzy Score Comparision

## 3.3 Jaccard Scores

### 3.3.1 Jaccard Scores between SPID & SPI

In [ ]:
from sklearn.metrics import jaccard_score

In [ ]:
j_df= pd.read_csv('Fuzzy_10k_Instr_in_SPID.csv')

In [ ]:
j_df.columns

In [ ]:
import pandas as pd
def jaccard_similarity(col1, col2):
    set1 = set(col1.split())
    set2 = set(col2.split())
    return len(set1.intersection(set2)) / float(len(set1.union(set2)))

j_df['jaccard_score'] = j_df.apply(lambda row: jaccard_similarity(row['spi_sort'], row['spid_sort']), axis=1)

In [ ]:
j_df.head(20)

Observation: The Jaccard Score and combination of fuzzy score can be used to identify bets match.

### 3.3.1 Jaccard Scores between SPID & PDS

In [ ]:
pds_j_df= pd.read_csv('Fuzzy_10k_PDS_in_SPID.csv')

In [ ]:
pds_j_df.columns

In [ ]:
import pandas as pd
def jaccard_similarity(col1, col2):
    set1 = set(col1.values.astype('U').split())
    set2 = set(col2.values.astype('U').split())
    return len(set1.intersection(set2)) / float(len(set1.union(set2)))

pds_j_df['jaccard_score'] = pds_j_df.apply(lambda row: jaccard_similarity(row['pds_sort'], row['spid_sort']), axis=1)

In [ ]:
pds_j_df.head(20)

## 3.4 Human Validation

### 3.4.1 Labeling Data

### 3.4.2 Confidence Matrix

In [ ]:
# Labelled data is needed as output

# 4 Performance

In [ ]:
#Metrics

In [ ]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.85)
t = time.time()-t1
print("SELFTIMED:", t)
Outputs:
  SELFTIMED: 0.19s

In [ ]:
Example Remark:
    
    'E-101' is '70%' similar to 'C-FE-102' after '2' 'Alphabet' Insertion(s) & '1' 'Numeric' Modification(s).

# 5 Business Conclusions

# 6 Rough

### 6.2 Data Pre-processing Unique DF

In [ ]:
for i in df.columns:
    print('{0} Column Count is {1}:'.format(i,df[i].count()))

### 1.6 Convert astype('str')

In [ ]:
data["PID"].apply(lambda x: str(x))
data['PID'].dtypes

data=data.astype(str)

for i in data.columns:
    data[i].apply(str)
    #data.astype({i:'string'})
    #data[i].values.astype('str')

### 6.6 Data Pre-processing- Clustering

In [ ]:
import scipy.cluster.hierarchy as hc
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

import scipy.cluster.hierarchy as hc
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

clust_pids = unique_df.drop(['SPI', 'PDS'], axis=1)

clust_spis = unique_df.drop(['PID','PDS'], axis=1)

clust_pdss = unique_df.drop(['SPI', 'PID'], axis=1)

clust_pids.head(3)

clust_pids.isnull().sum()

type(clust_pids)

clust_pids.head(3)


In [ ]:
### Clustering ChatGPT
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

corpus = [
    "The quick brown fox jumps over the lazy dog.",
    "The brown fox jumps over the brown dog.",
    "The red fox jumps over the brown dog.",
    "The brown cat jumps over the brown dog."
]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

k = 2  # number of clusters
kmeans = KMeans(n_clusters=k)
kmeans.fit(X)

for i in range(k):
    print("Cluster", i+1, ":")
    for j in range(len(corpus)):
        if kmeans.labels_[j] == i:
            print("-", corpus[j])
    print()


https://vitalflux.com/hierarchical-clustering-explained-with-python-example/

In [ ]:
#https://towardsdatascience.com/how-to-easily-cluster-textual-data-in-python-ab27040b07d8

def cluster_text(text):
    vectorizer = TfidfVectorizer(stop_words={'english'})
    X = vectorizer.fit_transform(text)

    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    Sum_of_squared_distances = []
    K = range(2,10)
    for k in K:
       km = KMeans(n_clusters=k, max_iter=200, n_init=10)
       km = km.fit(X)
       Sum_of_squared_distances.append(km.inertia_)
    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

    print('How many clusters do you want to use?')
    true_k = int(input())
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
    model.fit(X)

    labels=model.labels_
    clusters=pd.DataFrame(list(zip(text,labels)),columns=['title','cluster'])
    #print(clusters.sort_values(by=['cluster']))

    for i in range(true_k):
        print(clusters[clusters['cluster'] == i])
        
    return